#Convolutional Neural Network(UNET)

**Importing Libraries**

Load the dependencies and modules required for the rest of the script

In [0]:
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plot
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split

**Initialization of Variables**

In [0]:
width=160
height=120
i=0
taille_batch=50
nbr_entrainement=100
learning_rate=1E-3
nbr_mask=13
momentum=0.99 
tab_img=[]
tab_mask=[]
img_mask=[]

**Loading images and masks**

 https://www.kaggle.com/kumaresanmanickavelu/lyft-udacity-challenge

In [0]:
dir_img="CameraRGB/"
dir_mask="CameraSeg/"

In [0]:
for dir in ['/content/drive/My Drive/Colab Notebooks/dataset/lyft-udacity-challenge/resized/dataA/']:
  for file in os.listdir(dir+dir_img):
    i=i+1
    print(i)
    tab_img.append(cv2.imread(dir+dir_img+file)/255)
    img_mask=cv2.imread(dir+dir_mask+file)[:,:,2]
    img_mask_result=np.zeros(shape=( height,width,13), dtype=np.float32)
    img_mask_result[:,:,0][img_mask==1]=1.
    img_mask_result[:,:,1][img_mask==2]=1.
    img_mask_result[:,:,2][img_mask==3]=1.
    img_mask_result[:,:,3][img_mask==4]=1.
    img_mask_result[:,:,4][img_mask==5]=1.
    img_mask_result[:,:,5][img_mask==6]=1.
    img_mask_result[:,:,6][img_mask==7]=1.
    img_mask_result[:,:,7][img_mask==8]=1.
    img_mask_result[:,:,8][img_mask==9]=1.
    img_mask_result[:,:,9][img_mask==10]=1.
    img_mask_result[:,:,10][img_mask==11]=1.
    img_mask_result[:,:,11][img_mask==12]=1.
    img_mask_result[:,:,12][img_mask==13]=1.
    tab_mask.append(img_mask_result)
    if i==500:
      break

**Create the Placeholders**

In [0]:
ph_images=tf.placeholder(shape=(None, 120,160, 3), dtype=tf.float32,name='images') #3 Colors
ph_masks=tf.placeholder(shape=(None,120,160,13), dtype=tf.float32)  #13 Classes
ph_is_training=tf.placeholder_with_default(False, (), name='is_training')

In [0]:
tab_img=np.array(tab_img)
tab_mask=np.array(tab_mask)

In [0]:
train_images, test_images, train_labels, test_labels=train_test_split(tab_img, tab_mask, test_size=.1)


**You can create your own Normalization function or use "tf.layers.batch_normalization"**

In [0]:
#def normalisation(couche_prec):
    #mean, var=tf.nn.moments(couche_prec, [0])
    #scale=tf.Variable(tf.ones(shape=(np.shape(couche_prec)[-1])))
    #beta=tf.Variable(tf.zeros(shape=(np.shape(couche_prec)[-1])))
    #result=tf.nn.batch_normalization(couche_prec, mean, var, beta, scale, 0.001)
    #return result

**Convolution fonction**

In [0]:
def create_conv(couche_prec,taille_noyau, nbr_noyau,stride,training=False):
    # First convolution
    conv_W = tf.Variable(tf.truncated_normal(shape=(taille_noyau, taille_noyau, int(couche_prec.get_shape()[-1]), nbr_noyau))) 
    conv_b = tf.Variable(tf.zeros(nbr_noyau))
    conv   = tf.nn.conv2d(couche_prec, conv_W, strides=[1,stride,stride, 1], padding='SAME') + conv_b
    conv=tf.layers.batch_normalization(conv, training=training,momentum=momentum)
    return conv

**Deconvolution fonction**

In [0]:
def deconvolution(input, taille_noyau, nbr_conv, stride,training=False):
    w=tf.Variable(tf.random.truncated_normal(shape=(taille_noyau, taille_noyau, nbr_conv, int(input.get_shape()[-1])))) 
    b=np.zeros(nbr_conv)
    out_h=int(input.get_shape()[1])*stride
    out_w=int(input.get_shape()[2])*stride
    b_size=tf.shape(input)[0]
    result=tf.nn.conv2d_transpose(input, w, output_shape=[b_size, out_h, out_w, nbr_conv], strides=[1, stride, stride, 1], padding='SAME') #declaration de la géometrie de la couche de sortie
    result=result+ b
    result=tf.layers.batch_normalization(result, training=training,momentum=momentum)
    return result

**Building UNET Architecture**

The comments indicate the size of the images after each convolution to help you understand well.

In [0]:
C1 = create_conv(ph_images,3,16,1,ph_is_training) #C1 (160*120*16)
C1 = tf.nn.relu(C1)   # Activation: relu

C2 = create_conv(C1,3, 16,1,ph_is_training) #C2 (160*120*16)
C2 = tf.nn.relu(C2)

C3 = tf.nn.max_pool(C2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') # Pooling1

C3 = create_conv(C3,3, 32,1,ph_is_training) #C3 (80*60*32)
C3= tf.nn.relu(C3)

C4 = create_conv(C3,3, 32,1,ph_is_training) #C4 (80*60*32)
C4 = tf.nn.relu(C4)

C5= tf.nn.max_pool(C4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') # Pooling2

C5 = create_conv(C5,3, 64,1,ph_is_training) #C5 (40*30*64)
C5 = tf.nn.relu(C5)

C6 = create_conv(C5,3, 64,1,ph_is_training) #C6 (40*30*64)
C6 = tf.nn.relu(C6)

C7 = tf.nn.max_pool(C6, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') # Pooling3

C7 = create_conv(C7,3, 128,1,ph_is_training) #C7 (20*15*128)
C7 = tf.nn.relu(C7)

C8 = create_conv(C7,3, 128,1,ph_is_training) #C8 (20*15*128)
C8= tf.nn.relu(C8)



In [0]:
D1=deconvolution(C8,3,64,2,ph_is_training) #D1 
D1= tf.nn.relu(D1)
result=tf.concat((D1, C6), axis=3)
result=create_conv(result,3,64,1,ph_is_training) #C1 (40*30*64)
result = tf.nn.relu(result)

result=create_conv(result,3,64,1,ph_is_training) #C2 (40*30*64)
result = tf.nn.relu(result)

D2=deconvolution(result,3,64,2,ph_is_training) #D2
result= tf.nn.relu(D2)
result=tf.concat((D2, C4), axis=3)
result=create_conv(result,3,32,1,ph_is_training) #C3 (80*60*32)
result = tf.nn.relu(result)

result=create_conv(result,3,32,1,ph_is_training) #C4 (80*60*32)
result = tf.nn.relu(result)

D3=deconvolution(result,3,32,2,ph_is_training) #D3
result= tf.nn.relu(D3)
result=tf.concat((D3, C2), axis=3)
result=create_conv(result,3,16,1,ph_is_training) #C5 (160*120*16)
result = tf.nn.relu(result)

result=create_conv(result,3,16,1,ph_is_training) #C6 (160*120*16)
result = tf.nn.relu(result)

w=tf.Variable(tf.random.truncated_normal(shape=(1,1,int(result.get_shape()[-1]),nbr_mask)))
b=np.zeros(nbr_mask)
result= tf.nn.conv2d(result,w,strides=[1,1,1,1], padding='SAME')
result=result +b 
mask=tf.nn.sigmoid(result,name="sortie")


**Model training and optimization**

-For the loss function, we must use cross entropy with **Sigmoid** instead of **Softmax**, because the input is not a vector of probabilities but a single value of probability which equals 1 if the pattern is present and 0 otherwise.

-Likewise for the precision calculation, instead of **argmax** we use **round** which returns 1 if the probability is> 0.5 and 0 otherwise.

-The optimization method **AdamOptimizer** performs better than **GradientDescent**

In [0]:
loss=tf.nn.sigmoid_cross_entropy_with_logits(labels=ph_masks, logits=result) 
train=tf.train.AdamOptimizer(learning_rate).minimize(loss) 
accuracy=tf.reduce_mean(tf.cast(tf.equal(tf.round(mask), ph_masks), tf.float32)) 

In [0]:
saver=tf.train.Saver()
with tf.Session() as s:
  s.run(tf.global_variables_initializer())
  tab_train=[]
  tab_test=[]
  for id_entrainement in np.arange(nbr_entrainement):
      print("> Entrainement", id_entrainement)
      for batch in np.arange(0, len(train_images), taille_batch):
          s.run(train, feed_dict={
              ph_images: train_images[batch:batch+taille_batch],
              ph_masks: train_labels[batch:batch+taille_batch],
              ph_is_training: True
          })
      print("  entrainement OK")
      tab_accuracy_train=[]
      for batch in np.arange(0, len(train_images), taille_batch):
          p=s.run(accuracy, feed_dict={
              ph_images: train_images[batch:batch+taille_batch],
              ph_masks: train_labels[batch:batch+taille_batch]
          })
          tab_accuracy_train.append(p)
      print("  train:", np.mean(tab_accuracy_train))
      tab_accuracy_test=[]
      for batch in np.arange(0, len(test_images), taille_batch):
          p=s.run(accuracy, feed_dict={
              ph_images: test_images[batch:batch+taille_batch],
              ph_masks: test_labels[batch:batch+taille_batch]
          })
          tab_accuracy_test.append(p)
      print("  test :", np.mean(tab_accuracy_test))
      tab_train.append(1-np.mean(tab_accuracy_train))
      tab_test.append(1-np.mean(tab_accuracy_test))
  #Affichage des deux courbes de performance 
  plot.ylim(0, 1)
  plot.grid()
  plot.plot(tab_train, label="Train error")
  plot.plot(tab_test, label="Test error")
  plot.legend(loc="upper right")
  plot.show()
  resulat=s.run(mask, feed_dict={ph_images: test_images[0:taille_batch]})
  np.set_printoptions(formatter={'float': '{:0.3f}'.format})
  for image in range(taille_batch):
    print("image", image)
    print("sortie du réseau:", resulat[image], np.argmax(resulat[image]))
    print("sortie attendue :", test_labels[image], np.argmax(test_labels[image]))
    cv2_imshow(test_images[image]*255)
  #Sauvegarde du modèle
  saver.save(s, './drive/My Drive/modele')
